In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install patchify opencv-python tensorflow numpy scikit-learn --upgrade

In [ ]:
import numpy as np
import cv2
import os
import shutil
import matplotlib.pyplot as plt
from PIL import Image
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import random

In [ ]:
minmaxscaler = MinMaxScaler()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
for path, subdirs, files in os.walk('/kaggle/input'):
  dir_name = path.split(os.path.sep)[-1]
  print(dir_name)
  if dir_name == 'images':
    images = os.listdir(path)
    print(path)
    for i, image_name in enumerate(images):
      if(image_name.endswith('jpg')):
        print(image_name)
        a = True

In [ ]:
image = cv2.imread('/kaggle/input/semantic-segmentation-of-aerial-imagery/Semantic segmentation dataset/Tile 1/images/image_part_001.jpg',1)

In [ ]:
type(image)

In [ ]:
type(Image.fromarray(image))


In [ ]:
image
image.shape

In [ ]:
image_patch_size = 256

In [ ]:
image_patches = patchify(image,(image_patch_size, image_patch_size,3),step = image_patch_size)

In [ ]:
image_x = image_patches[0,0,:,:]
images_y = minmaxscaler.fit_transform(image_x.reshape(-1, image_x.shape[-1])).reshape(image_x.shape)

In [ ]:
print(image_patches.shape)

In [ ]:
image_dataset = []
mask_dataset = []
for image_type in ['images' , 'masks']:
  if image_type == 'images':
    image_ex = 'jpg'
  elif image_type == 'masks':
     image_ex = 'png'
  for tile_id in range(1,9):
    for image_id in range(1,10):
      image = cv2.imread(f'/kaggle/input/semantic-segmentation-of-aerial-imagery/Semantic segmentation dataset/Tile {tile_id}/{image_type}/image_part_00{image_id}.{image_ex}',1)
      if image is not None:
        if image_type == 'masks':
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #print(image.shape)
        size_x = (image.shape[1]//image_patch_size)*image_patch_size
        size_y = (image.shape[0]//image_patch_size)*image_patch_size
        #print("{} --- {} - {}".format(image.shape, size_x, size_y))
        image = Image.fromarray(image)
        image = image.crop((0,0, size_x, size_y))
        #print("({},  {})".format(image.size[0],image.size[1]))
        image = np.array(image)
        patched_images = patchify(image, (image_patch_size, image_patch_size, 3), step=image_patch_size)
        #print(len(patched_images))
        for i in range(patched_images.shape[0]):
          for j in range(patched_images.shape[1]):
            if image_type == 'images':
              individual_patched_image = patched_images[i,j,:,:]
              #print(individual_patched_image.shape)
              individual_patched_image = minmaxscaler.fit_transform(individual_patched_image.reshape(-1, individual_patched_image.shape[-1])).reshape(individual_patched_image.shape)
              individual_patched_image = individual_patched_image[0]
              #print(individual_patched_image.shape)
              image_dataset.append(individual_patched_image)
            elif image_type == 'masks':
              individual_patched_mask = patched_images[i,j,:,:]
              individual_patched_mask = individual_patched_mask[0]
              mask_dataset.append(individual_patched_mask)


In [ ]:
print(len(image_dataset))
print(len(mask_dataset))

In [ ]:
mask_dataset[0]

In [ ]:
image_dataset = np.array(image_dataset)
mask_dataset = np.array(mask_dataset)

In [ ]:
random_image_num = random.randint(0,len(image_dataset))

plt.figure(figsize = (15,10))
plt.subplot(121)
plt.imshow(image_dataset[random_image_num])
plt.subplot(122)
plt.imshow(mask_dataset[random_image_num])

In [ ]:
class_building = '#3C1098'
class_building = class_building.lstrip("#")
class_building = np.array(tuple(int(class_building[i:i+2], 16) for i in (0,2,4)))
print(class_building)

class_land = '#8429F6'
class_land = class_land.lstrip("#")
class_land = np.array(tuple(int(class_land[i:i+2], 16) for i in (0,2,4)))
print(class_land)

class_vegetation = '#FEDD3A'
class_vegetation = class_vegetation.lstrip("#")
class_vegetation = np.array(tuple(int(class_vegetation[i:i+2], 16) for i in (0,2,4)))
print(class_vegetation)

class_road = '#6EC1E4'
class_road = class_road.lstrip("#")
class_road = np.array(tuple(int(class_road[i:i+2], 16) for i in (0,2,4)))
print(class_road)

class_water = '#E2A929'
class_water = class_water.lstrip("#")
class_water = np.array(tuple(int(class_water[i:i+2], 16) for i in (0,2,4)))
print(class_water)

class_unlabeled = '#9B9B9B'
class_unlabeled = class_unlabeled.lstrip("#")
class_unlabeled = np.array(tuple(int(class_unlabeled[i:i+2], 16) for i in (0,2,4)))
print(class_unlabeled)

In [ ]:
mask_dataset.shape[0]

In [ ]:
label = individual_patched_mask

In [ ]:
def rgb_to_label(label):
  label_segment = np.zeros(label.shape, dtype=np.uint8)
  label_segment[np.all(label == class_water, axis=-1)] = 0
  label_segment[np.all(label == class_land, axis=-1)] = 1
  label_segment[np.all(label == class_road, axis=-1)] = 2
  label_segment[np.all(label == class_building, axis=-1)] = 3
  label_segment[np.all(label == class_vegetation, axis=-1)] = 4
  label_segment[np.all(label == class_unlabeled, axis=-1)] = 5
  #print(label_segment)
  label_segment = label_segment[:,:,0]
  #print(label_segment)
  return label_segment

In [ ]:
labels = []
for i in range(mask_dataset.shape[0]):
  label = rgb_to_label(mask_dataset[i])
  labels.append(label)


In [ ]:
print(len(labels))

In [ ]:
labels = np.array(labels)

In [ ]:
labels[0]

In [ ]:
labels = np.expand_dims(labels,axis=3)

In [ ]:
labels[1][2]

In [ ]:
np.unique(labels)

In [ ]:
random_image_num = random.randint(0,len(image_dataset))

plt.figure(figsize = (15,10))
plt.subplot(121)
plt.imshow(image_dataset[random_image_num])
plt.subplot(122)
plt.imshow(labels[random_image_num][:,:,0])

In [ ]:
training_set = image_dataset

In [ ]:
import tensorflow as tf


In [ ]:
from tensorflow.keras.utils import to_categorical


In [ ]:
labels_categorical_data = to_categorical(labels,num_classes = len(np.unique(labels)))

In [ ]:
labels_categorical_data.shape

In [ ]:
training_set.shape


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(training_set,labels_categorical_data,test_size = 0.2,random_state = 100)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
image_height = x_train.shape[1]
image_width = x_train.shape[2]
image_channels = x_train.shape[3]
image_classes = y_train.shape[3]

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from tensorflow.keras.layers import concatenate, BatchNormalization, Dropout, Lambda

In [ ]:
from tensorflow.keras import backend as K

In [ ]:
def jaccard_coef(y_true, y_pred):
  y_true_flatten = K.flatten(y_true)
  y_pred_flatten = K.flatten(y_pred)
  intersection = K.sum(y_true_flatten * y_pred_flatten)
  final_coef_value = (intersection + 1.0) / (K.sum(y_true_flatten) + K.sum(y_pred_flatten) - intersection + 1.0)
  return final_coef_value

In [ ]:
def multi_unet_model(n_classes=5,
                     image_height=256,
                     image_width=256,
                     image_channels=1):

    # -------- Encoder --------
    inputs = Input(shape=(image_height, image_width, image_channels))

    c1 = Conv2D(16,  (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(inputs)
    c1 = Dropout(0.2)(c1)
    c1 = Conv2D(16,  (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32,  (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(p1)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(32,  (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64,  (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64,  (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(p4)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(256, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c5)

    # -------- Decoder --------
    u6 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding="same")(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c6)

    u7 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same")(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c7)

    u8 = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding="same")(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(u8)
    c8 = Dropout(0.2)(c8)
    c8 = Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c8)

    u9 = Conv2DTranspose(16, (3, 3), strides=(2, 2), padding="same")(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(u9)
    c9 = Dropout(0.2)(c9)
    c9 = Conv2D(16, (3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(c9)

    outputs = Conv2D(n_classes, (1, 1), activation="softmax")(c9)

    model = Model(inputs=inputs, outputs=outputs)
    return model


In [ ]:
metrics = ['accuracy', jaccard_coef]

In [ ]:
def get_model():
  return multi_unet_model(n_classes=image_classes ,
                          image_height=image_height,
                          image_width=image_width,
                          image_channels=image_channels)


In [ ]:
model = get_model()

In [ ]:
model.get_config()

In [ ]:
weights = [0.166,0.166,0.166,0.166,0.166,0.166]


In [ ]:
!pip uninstall tensorflow -y

In [ ]:
!pip install tensorflow==2.15.0

In [ ]:
!pip install segmentation-models

In [ ]:
import os
os.environ['SM_FRAMEWORK'] = 'tf.keras'  # Set environment variable for segmentation_models
import segmentation_models as sm

# Verify framework
print(sm.__version__)


In [ ]:
dice_loss = sm.losses.DiceLoss(class_weights = weights)

In [ ]:
focal_loss  = sm.losses.CategoricalFocalLoss()

In [ ]:
total_loss = dice_loss + (1*focal_loss)

In [ ]:
tf.keras.backend.clear_session()


In [ ]:
model.compile(optimizer ='adam',
              loss = total_loss,
             metrics = metrics)

In [ ]:
model.summary()

In [ ]:
model_history = model.fit(x_train,y_train,
                         batch_size=16,
                         verbose = 1,
                         epochs = 10,
                         validation_data=(x_test,y_test),
                         shuffle = False)

In [ ]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']
epochs = range(1,len(loss)+1 )
plt.plot(epochs,loss,'y', label = 'training_loss')
plt.plot(epochs,val_loss,'r', label = 'validation_loss') 
plt.title('Training loss vs. Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
jaccard_coef = model_history.history['jaccard_coef']
val_jaccard_coef = model_history.history['val_jaccard_coef']
epochs = range(1,len(jaccard_coef )+1 )
plt.plot(epochs,jaccard_coef,'y', label = 'Training IoU')
plt.plot(epochs,val_jaccard_coef,'r', label = 'Validation IoU') 
plt.title('Training IoU vs. Validation IoU')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_pred

In [ ]:
y_pred_argmax = np.argmax(y_pred,axis=3)

In [ ]:
y_pred_argmax

In [ ]:
len(y_pred_argmax)

In [ ]:
y_test_argmax = np.argmax(y_test,axis=3)

In [ ]:
y_test_argmax

In [ ]:
test_img_num = 10
test_img = x_test[test_img_num]
ground_truth_image = y_test_argmax[test_img_num]
 
test_image_input = np.expand_dims(test_img,0)

prediction = model.predict(test_image_input)
predicted_image = np.argmax(prediction,axis=3)


In [ ]:
predicted_image

In [ ]:
predicted_image = predicted_image[0,:,:]
predicted_image

In [ ]:
plt.figure(fig_size =(14,8))
plt.subplot(231)
plt.title('Original Test Image')
plt.imshow(test_img)

plt.subplot(232)
plt.title('Ground Truth Image')
plt.imshow(ground_truth_image)

plt.subplot(232)
plt.title('Predicted Image')
plt.imshow(predicted_image)



In [ ]:
plt.imshow(test_img)

In [ ]:
plt.imshow(ground_truth_image)

In [ ]:
plt.imshow(predicted_image)

In [ ]:
model.save('satellite_seg_model.h5')